In [2]:
import torch

torch.__version__

'2.0.0'

In [3]:
x = torch.empty(5,3)
x

tensor([[2.1707e-18, 7.0952e+22, 1.7748e+28],
        [1.8176e+31, 7.2708e+31, 5.0778e+31],
        [3.2608e-12, 1.7728e+28, 7.0367e+22],
        [2.1715e-18, 8.1725e+20, 5.3482e+22],
        [3.3091e+21, 1.3295e+22, 1.6517e-04]])

In [5]:
x = torch.rand(5,3)
x

tensor([[0.5604, 0.3315, 0.8753],
        [0.8215, 0.2825, 0.8272],
        [0.6006, 0.9827, 0.7480],
        [0.0241, 0.1480, 0.6545],
        [0.8559, 0.9843, 0.3728]])

In [6]:
x = torch.zeros(5,3, dtype=torch.long)
x


tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [7]:
x = torch.tensor([5.5,3])
x

tensor([5.5000, 3.0000])

In [8]:
x.size()

torch.Size([2])

In [9]:
x = torch.rand(4,4)
y = x.view(16)
z = x.view(-1, 8)
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


In [11]:
a = torch.ones(5)
b = a.numpy()
a
b

array([1., 1., 1., 1., 1.], dtype=float32)

In [14]:
import numpy as np

a = np.ones(5)
b = torch.from_numpy(a)
b

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)

反向传播计算

In [15]:
# 方法1
x = torch.randn(3,4,requires_grad=True)
x

tensor([[-0.6494, -0.4062,  0.5814, -0.3304],
        [ 0.5446,  0.3968, -0.3324,  1.2964],
        [-0.6384,  0.4050, -1.1050, -0.3918]], requires_grad=True)

In [26]:
b = torch.randn(3, 4, requires_grad=True)
b.grad

In [27]:
t = x + b
t

tensor([[ 0.2559, -0.3608,  0.0497, -0.0435],
        [-0.2808, -0.3751, -1.8405,  0.9416],
        [ 1.1698,  0.6444, -1.7483, -1.5815]], grad_fn=<AddBackward0>)

In [29]:
y = t.sum()
y

tensor(-3.1692, grad_fn=<SumBackward0>)

In [28]:
b.grad

In [30]:
x = torch.rand(1)
b = torch.rand(1, requires_grad=True)
w = torch.rand(1, requires_grad=True)
y = w * x
z = y + b

In [31]:
x.requires_grad, b.requires_grad, w.requires_grad, y.requires_grad

(False, True, True, True)

In [35]:
z.backward(retain_graph=True)

In [36]:
w.grad

tensor([0.5015])

In [37]:
b.grad

tensor([2.])

简单的线性回归模型

In [38]:
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1) # -1是自适应元素大小，从1*11 转成11*1
x_train.shape


(11, 1)

In [39]:
y_values = [2*i for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

(11, 1)

In [40]:
import torch
import torch.nn as nn

In [46]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

In [47]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [48]:
# 指定参数进行训练
# 指定损失函数
epochs = 1000  # 训练1000轮
learning_rate = 0.01  #学习率
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss() #损失函数，均方误差


In [50]:
for epoch in range(epochs):
    epoch += 1
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)

    #梯度每次迭代计算得清除缓存
    optimizer.zero_grad()
    # 前向传播
    outputs = model.forward(inputs)

    #计算损失
    loss = criterion(outputs, labels)
    # 反向传播, 求梯度
    loss.backward()
    # 更新权重参数
    optimizer.step()  #step()函数会自动帮你完成梯度计算和参数值更新
    if epoch % 50 == 0:
        print(f"epoch {epoch}, loss {loss.item()}")
    

epoch 50, loss 0.005260835401713848
epoch 100, loss 0.003000580007210374
epoch 150, loss 0.001711421413347125
epoch 200, loss 0.000976134673692286
epoch 250, loss 0.0005567456246353686
epoch 300, loss 0.0003175561432726681
epoch 350, loss 0.00018112001998815686
epoch 400, loss 0.000103305748780258
epoch 450, loss 5.892197441426106e-05
epoch 500, loss 3.360401387908496e-05
epoch 550, loss 1.9168017388437875e-05
epoch 600, loss 1.0932187251455616e-05
epoch 650, loss 6.235657110664761e-06
epoch 700, loss 3.555977855285164e-06
epoch 750, loss 2.0284510355850216e-06
epoch 800, loss 1.1567503861442674e-06
epoch 850, loss 6.598575055249967e-07
epoch 900, loss 3.76191565010231e-07
epoch 950, loss 2.1473624656209722e-07
epoch 1000, loss 1.2245097025243012e-07


In [51]:
# 模型测试
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[6.5086503e-04],
       [2.0005572e+00],
       [4.0004635e+00],
       [6.0003695e+00],
       [8.0002756e+00],
       [1.0000182e+01],
       [1.2000088e+01],
       [1.3999994e+01],
       [1.5999901e+01],
       [1.7999807e+01],
       [1.9999714e+01]], dtype=float32)

In [53]:
y_train

array([[ 0.],
       [ 2.],
       [ 4.],
       [ 6.],
       [ 8.],
       [10.],
       [12.],
       [14.],
       [16.],
       [18.],
       [20.]], dtype=float32)

In [55]:
# 模型保存
torch.save(model.state_dict(), "LinearModel.pkl")


In [56]:
# 加载模型
model.load_state_dict(torch.load("LinearModel.pkl"))

<All keys matched successfully>